In [ ]:
import os
home = os.getcwd()
os.chdir(home + "/LFM/imaging")


In [ ]:
import os
os.getcwd()

In [ ]:
from reconstruction import reconstruct_vols_from_imgs
from i_o import Paths
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os, glob, h5py
import importlib, pathlib
from daio.h5 import lazyh5
from video import create_projection_image, recording_to_overlay_preview, showvid, get_lenses

cmap = "gray"

In [ ]:
# pn_rec = r"Y:/hpc/r/lfm2025/recordings"
pn_rec = r"~/hpc-r/lfm2025/recordings"
includes = ['']
excludes = ["bg"]

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_rec, '*'))), reverse=True) if os.path.exists(os.path.join(pn_rec, rec, 'data.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

In [ ]:
# pn_psf = r"Y:/hpc/r/lfm2025/psfs"
pn_psfs = r"~/hpc-rw/lfm/psfs"
includes = ['']
excludes = []

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_psfs, '*'))), reverse=True) if os.path.exists(os.path.join(pn_psfs, rec, 'psf.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

In [ ]:
pn_bg = r"~/hpc-rw/lfm/bg"
for file in pathlib.Path(os.path.expanduser(pn_bg)).iterdir():
    if file.is_file():
        print(file.name)

In [ ]:
pn_psfs = r"~/hpc-rw/lfm/psfs"
pn_rec = r"~/hpc-r/lfm2025/recordings"
pn_bg = r"~/hpc-rw/lfm/bg"
url_home = r"/home/lubo12/"
pn_out = r"~/hpc-rw/lfm/results"

dataset_name = "20250701_1605_LB_ZF_v552_f6_40fps_1"
psf_name = "20250701_1216_PSF_LB_noap_1"
bg_name = "20250701_1651_LB_bg_40fps.npy"

paths = Paths(dataset_name=dataset_name,
              psf_name = psf_name,
              bg_name=bg_name,
              pn_bg= pn_bg,
              pn_rec = pn_rec,
              pn_psfs=pn_psfs,
              pn_out=pn_out,
              url_home=url_home,
              )

In [ ]:
psf_f = lazyh5(paths.psf)
crop = np.array(psf_f["crop"])
mask = np.array(psf_f["circle_mask"][crop[0]:crop[1],crop[2]:crop[3]])
psf = np.array(psf_f["psf"])
zpos=np.array(psf_f["z_positions"])

print(psf.shape)

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.imshow(create_projection_image(psf, scalebar=200,zpos=zpos, text= "raw PSF", text_size=4), cmap='gray')
# ax[1].imshow(create_projection_image(psf["psf"]-bg,np.max,pad=10), cmap='binary')

In [ ]:
print(og_crop, og_crop[1]-og_crop[0],og_crop[3]-og_crop[2])
better_crop = os.path.expanduser(os.path.join(pn_psfs, "20250509_1646_PSF_LB_redFB_1_30K_wo_coverslip_1","psf.h5"))
with h5py.File(better_crop,"r") as f:
    crop = list(f["crop"])
    og_mask=np.array(f["circle_mask"])
print(crop, crop[1]-crop[0],crop[3]-crop[2])
plt.imshow(og_mask[crop[0]:crop[1],crop[2]:crop[3]])


In [ ]:
crop[0]=crop[0]-1
crop[2]=crop[2]-1
print(crop, crop[1]-crop[0],crop[3]-crop[2])
mask = og_mask[crop[0]:crop[1],crop[2]:crop[3]]

In [ ]:
vmax=100

fig, ax = plt.subplots(1,2,figsize = (14,7))
bg = lazyh5(paths.bg)["data"][crop[0]:crop[1],crop[2]:crop[3]]
data = lazyh5(paths.raw)
ims = data["data"][25:50,crop[0]:crop[1],crop[2]:crop[3]]
im = ims[0]

ax[0].imshow(im, cmap = "gray",vmax=vmax)
ax[1].imshow((im-bg)*mask, cmap="gray",vmax=vmax)
print(dataset_name)

In [ ]:
cx, cy, radii, masks = get_lenses(og_mask, radius_range=np.arange(525,535))

In [ ]:
cx=np.array(psf_f["ml_cx"])
cy=np.array(psf_f["ml_cy"])
radii=np.array(psf_f["ml_radii"])
masks=np.array(psf_f["ml_masks"])


In [ ]:
import video
importlib.reload(video)
from video import recording_to_overlay_preview, get_lenses, img_to_overlay_preview, showvid, get_projection_image
fn_vid, fn_file ,fn_dff_vid, fn_dff_file = recording_to_overlay_preview(paths,
                                                      cx,cy,padxy=300,masks=masks, 
                                                      fps=40, 
                                                      avg_method="median", median_avg_window =40, 
                                                      dff=True,
                                                      vmin=0,vmax=100,absolute_limits=False)
showvid(fn_vid)

In [ ]:
import reconstruction
importlib.reload(reconstruction)
from reconstruction import reconstruct_vols_from_imgs
img_idx = [25,26,1]
# print(f"PSF zspacing: {np.abs(np.diff(psf["z_positions"][::2])).mean()*1000} um")
objs, mips, losses, kwargs = reconstruct_vols_from_imgs(paths,
                                                        img_idx=img_idx,
                                                        crop = crop,
                                                        roi_size=550,
                                                        loss_threshold=0,
                                                        reuse_prev_vol = False,
                                                        psf_downsample = [40,-30,1],
                                                        OTF_normalize=True,
                                                        OTF_clip=False,
                                                        img_subtract_bg=True,
                                                        img_mask=True,
                                                        plot_decon=True,
                                                        fully_batched=False,
                                                        vmin=0,
                                                        vmax=50,
                                                        absolute_limits=False
                                                        )
fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.imshow(mips[0,-1,:,:],cmap="gray")

In [ ]:
frame = 0

vmin=0
vmax=5
absolute_limits=False
_zpos = zpos[kwargs["psf_downsample"][0]:kwargs["psf_downsample"][1]:kwargs["psf_downsample"][2]] if kwargs["psf_downsample"] is not None else zpos


fig, ax = plt.subplots(1,1,figsize=(14,14))
ax.imshow(create_projection_image(objs[0], vmin=vmin, vmax=vmax, absolute_limits=absolute_limits, zpos = _zpos, scalebar=200, text = f"f{np.arange(*img_idx)[frame]}",),cmap = "gray")

title = f"f{np.arange(*img_idx)[frame]}_roi{kwargs["roi_size"]}_img{"-bg" if kwargs["img_subtract_bg"] else ""}{"_masked"if kwargs["img_mask"] else ""}_vmin{vmin}_vmax{vmax}{"_al" if absolute_limits else ""}"
otf_name = f"OTF_{paths.psf_name}_pad{kwargs["xy_pad"]}{"_clip" if kwargs["OTF_clip"] else ""}{"_norm" if kwargs["OTF_normalize"] else ""}{f'_dwn{kwargs["psf_downsample"]}' if kwargs["psf_downsample"] is not None else ''}"

ax.set_title(paths.dataset_name+"_"+title +"\n"+otf_name)
os.mkdir(os.path.join(paths.pn_outrec,"testmip")) if not os.path.exists(os.path.join(paths.pn_outrec,"testmip")) else None
fig.savefig(os.path.join(paths.pn_outrec,"testmip",f"{title}_{otf_name}.png"))

In [ ]:
kwargs["write_mip_video"]=False
kwargs

In [ ]:
import slurm
importlib.reload(slurm)
from slurm import PythonExecutorSLURM
import reconstruction
importlib.reload(reconstruction)
from reconstruction import reconstruct_vols_from_imgs_parallel
# kwargs, save_fn, vid_fn = reconstruct_vols_from_imgs_parallel(paths,img_idx=(3020,3025,1),write_mip_video=True,**kwargs)
# print(save_fn,"\n",vid_fn)


In [ ]:
email = 'lukas.born@charite.de' 
#cmd_prefix = ['ssh', 'lubo12@s-sc-frontend1'] # if outside HPC (add your user name), otherwise: 
cmd_prefix = [] #if inside HPC
from slurm import PythonExecutorSLURM
slex = PythonExecutorSLURM(job_path=paths.pn_outrec, job_name='deconv'+paths.dataset_name, conda_env='lfm1',
                                     time="0-12:00", partition="pgpu", gres="gpu:4", cpus_per_task=16,mem='128G',ntasks=1,nodes=1,exclude="s-sc-pgpu03",
                                     cmd_prefix=cmd_prefix, mail_user=email, mail_type='ALL,TIME_LIMIT_90')

In [ ]:
job_id = slex.submit(reconstruct_vols_from_imgs_parallel, paths, **kwargs)

In [ ]:
result = lazyh5(paths.deconvolved)
result

In [ ]:
result.deconvolution_params["OTF"]

In [ ]:
import video
importlib.reload(video)
from video import recording_to_video
video_fn = recording_to_video(paths, fps=40, vmax=100, absolute_limits=False)